# Eleman Vektörlerinin Oluşturulması

Bu bölümde eleman vektörlerini oluşturacağız. Bir önceki ders 7b'de bu vektörleri denklem (7)'de $\mathbf{f}$ olarak  ifade etmiştik. Bu vektörleri tekrar yazalım:

$$ \mathbf{f} = \int_{\partial \Omega}\big(\mathbf{N}k\nabla T \big) \cdot \mathbf{n} dS  + \int_{\Omega} \mathbf{N} \big(h_z T_{\infty})  d\Omega = \mathbf{f}_1 + \mathbf{f}_2   \tag{1}$$ 

Denklem (1)'de görüldüğü üzere 1. ifadenin integrali geometri sınırları boyunca, 2. ifadenin integrali ise geometrinin kendisi üzerinde uygulanıyor. 

Burada yine başlangıç olarak 3. elemandan başlayalım. Ders 7c'de tanımladığımız şekil fonksiyonlarını ve türevlerini tekrar tanımlayalım:

In [1]:
import sympy as sp
import numpy as np
from scipy import integrate
from sympy.utilities import lambdify

xi = sp.Symbol("xi")
eta = sp.Symbol("eta")
x_ = sp.symbols("x1,x2,x3,x4")
y_ = sp.symbols("y1,y2,y3,y4")
şekil = sp.Matrix([(1+xi)*(1+eta)/4,
                   (1-xi)*(1+eta)/4,
                   (1-xi)*(1-eta)/4,
                   (1+xi)*(1-eta)/4])

In [2]:
nokta_sayısı = 4

gradyen_xi = sp.diff(şekil, xi)
gradyen_eta = sp.diff(şekil, eta)

gradyen = sp.Matrix([gradyen_eta, gradyen_xi])

gradyen = gradyen.reshape(2,4)

c_şekil = sp.Matrix([0, 0])
for i in range(nokta_sayısı):
    c_şekil[0] += x_[i] * şekil[i]
    c_şekil[1] += y_[i] * şekil[i]
J_sym = c_şekil.jacobian([xi, eta]).T

3 numaralı elemanı tekrar göz önüne getirelim:

<img src="Resimler/Ders7/7d_ElemanMesh.png" align="center" width= "600"/>

Ardından elemanın koordinatlarını tekrar tanımlayıp jakobyeni hesaplayalım:

In [3]:
x_koord = [1,0,0,1] 
y_koord = [2,2,1,1]

J = J_sym
for nokta in range(nokta_sayısı):
    J = J.subs(x_[nokta],x_koord[nokta]) 
    J = J.subs(y_[nokta],y_koord[nokta])
J

Matrix([
[1/2,   0],
[  0, 1/2]])

### Birinci integralin hesaplanması

Öncelikle birinci integrale göz atalım:

$$ \int_{\partial \Omega}\big(\mathbf{N}k\nabla T \big) \cdot \mathbf{n} \ dS = \int_{\partial \Omega}\mathbf{N}k\ \frac{\partial T}{\partial \mathbf{n}} \ dS. \tag{2} $$

Bu denklemde $k \partial T/\partial\mathbf{n}$ değeri, geometrinin sınırından gerçekleşen ısı transferi hızına ($\bar{q}$) eşittir. Bulduğumuz bu integrali hesaplarken ders 7b'deki 11 ve 12'inci denklemleri kullanacağız. Öncelikle dikey kenarlardaki sınır koşulunu temsil eden vektörü hesaplayacağımız özdeşliği yazarsak:

$$\int_{-1}^{1}
\begin{bmatrix}
N_1 \\ N_2 \\ N_3 \\ N_4
\end{bmatrix}
k 
\frac{\partial T}{\partial \mathbf{n}}(1, \eta)
\sqrt{
\Big(\frac{\partial x}{\partial \xi}\Big)^2+
\Big(\frac{\partial y}{\partial \xi}\Big)^2}
d \eta
\tag{3}
$$

ifadesini elde ederiz. Görüldüğü üzere integrasyon, sadece $\eta$ ekseninde gerçekleşmektedir, çünkü $\xi=1$ olarak sabit tutularak hesaplanmaktadır. Aynı şekilde yatay kenarlardaki sınır koşulunu temsil eden vektörün hesaplanacağı denklem

$$\int_{-1}^{1}
\begin{bmatrix}
N_1 \\ N_2 \\ N_3 \\ N_4
\end{bmatrix}
k 
\frac{\partial T}{\partial \mathbf{n}}(\xi, 1)
\sqrt{
\Big(\frac{\partial x}{\partial \eta}\Big)^2+
\Big(\frac{\partial y}{\partial \eta}\Big)^2}
d \xi
\tag{4}
$$

olarak belirtilir. Burada da farkedileceği üzere integrasyon $\xi$ ekseninde $\eta=1$ tutularak hesaplanmaktadır.

Son olarak $k \partial T/\partial\mathbf{n}$ ifadesi her iki eksen için ayrı ayrı yazılabilir. 3. eleman için bu değerleri nokta sırasına uygun şekilde $\xi$ ve $\eta$ eksenleri için ayrı ayrı yazarsak:

In [4]:
q_1 = -500 # W/m2
q_2 = -1000 # W/m2

q_eta = sp.Matrix([q_1,q_1,0,0])
q_xi = sp.Matrix([q_2,0,0,q_2])

matrislerini elde ederiz. Şimdi denklem (3) ve denklem (4)'ü hesaplayacağımız denklemleri tanımlayabiliriz:

In [5]:
f_e_1_sym_dikey = şekil.multiply_elementwise(q_xi)*sp.sqrt(J[0,0]**2+J[0,1]**2)
f_e_1_sym_yatay = şekil.multiply_elementwise(q_eta)*sp.sqrt(J[1,0]**2+J[1,1]**2)

f_e_1_sym_dikey = f_e_1_sym_dikey.subs(xi,1) 
f_e_1_sym_yatay = f_e_1_sym_yatay.subs(eta,1) 

Artık dört nokta için de sınır koşulu değerlerini oluşturan vektörü hesaplayabiliriz:

In [6]:
f_e_1 = np.zeros((nokta_sayısı,1))

for i in range(nokta_sayısı):
    dikey_fonksiyon = lambdify(eta, f_e_1_sym_dikey[i], 'math' )
    yatay_fonksiyon = lambdify( xi, f_e_1_sym_yatay[i], 'math' )
    f_e_1[i] += integrate.quad(dikey_fonksiyon, -1, 1)[0]
    f_e_1[i] += integrate.quad(yatay_fonksiyon, -1, 1)[0]
    
f_e_1

array([[-750.],
       [-250.],
       [   0.],
       [-500.]])

### İkinci integralin hesaplanması  

Denklem (1)'deki 2. terim olan

$$ \mathbf{f}_2 = \int_{\Omega} \mathbf{N} \big(h_z T_{\infty})  d\Omega $$

ifadesinin hesaplanması 1. terime kıyasla daha kolaydır. Bu ıfadeyi de yine izoparametrik eşleme ile yazarsak:

$$ \int_{\Omega} \mathbf{N} \big(h_z T_{\infty})  d\Omega = 
\int_{\Omega}
h_z T_{\infty}
\begin{bmatrix}
N_1 \\ N_2 \\ N_3 \\ N_4
\end{bmatrix}
d\Omega
=
\int_{-1}^1\int_{-1}^1
h_z T_{\infty}
\begin{bmatrix}
N_1 \\ N_2 \\ N_3 \\ N_4
\end{bmatrix}
det(\mathbf{J}) d\eta d\xi
\tag{5}
$$

denklemini elde ederiz. Bu denklemi yine *sympy* ile yazalım:

In [7]:
h = 10 # W/m2K
t_z = 0.05 #m
h_z = 2*h/t_z
T_sonsuz = 30 # C

f_e_2_sym = h_z*T_sonsuz*şekil*J.det()
f_e_2_sym

Matrix([
[750.0*(eta + 1)*(xi + 1)],
[750.0*(1 - xi)*(eta + 1)],
[750.0*(1 - eta)*(1 - xi)],
[750.0*(1 - eta)*(xi + 1)]])

Şimdi bu element için taşınımla ısı transferini temsil eden vektörü hesaplayabiliriz:

In [8]:
f_e_2 = np.zeros((nokta_sayısı,1))

for i in range(nokta_sayısı):
    taşınım_fonksiyon = lambdify( (xi,eta), f_e_2_sym[i], 'math' )
    f_e_2[i] += integrate.nquad(taşınım_fonksiyon, [[-1, 1],[-1, 1]])[0]

f_e_2

array([[3000.],
       [3000.],
       [3000.],
       [3000.]])

## Sonuç

Tüm bu işlemler ile 3. elemanın eleman iletim ve eleman taşınım matrislerinden sonra sınır koşulu vektörünü ve taşınım vektörünü hesaplamış olduk. Artık tüm sistemi çözümlemek için yapmamız gereken şey, bu işlemleri her bir eleman için tekrarlayıp global matris ve vektörleri oluşturmaktır. Ardından oluşturduğumuz matris sistemini çözümleyerek sıcaklık değerlerini elde edebiliriz. Bir sonraki derste bu aşamayı ele alacağız.